# 0.코딩준비

```
참조 예제 : https://tykimos.github.io/2017/04/09/RNN_Layer_Talk/ (LSTM 설명)
참조 예제 : https://bit.ly/2HQ9Q4J (수업시간 참조 예제)
악보 처리용 프로그램 추가 : https://musescore.org (MuseScore3인스톨)
악보를 보기 위한 setting :
us=music21.environment.UserSettings()
us["musescoreDirectPNGPath"]="C:/Program Files/MuseScore 3/bin/MuseScore3.exe”
pip install music21
```

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import tensorflow.keras.utils as utils
import os
from tensorflow.keras.callbacks import Callback #콜백객체
import matplotlib.pyplot as plt
import music21
us=music21.environment.UserSettings()
us["musescoreDirectPNGPath"]="C:/Program Files/MuseScore 3/bin/MuseScore3.exe"

# 1. 데이터 준비하기

In [ ]:
seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

print("length of seq: {0}".format(len(seq)))

In [ ]:
note_seq = ' '.join(seq)

m = music21.converter.parse("2/4 " + note_seq, format='tinyNotation')

m.show("midi")

In [ ]:
m.show()

In [ ]:
code2idx = {'c4': 0, 'd4': 1, 'e4': 2, 'f4': 3, 'g4': 4, 'a4': 5, 'b4': 6,
            'c8': 7, 'd8': 8, 'e8': 9, 'f8': 10, 'g8': 11, 'a8': 12, 'b8': 13}

idx2code = {0: 'c4', 1: 'd4', 2: 'e4', 3: 'f4', 4: 'g4', 5: 'a4', 6: 'b4',
            7: 'c8', 8: 'd8', 9: 'e8', 10: 'f8', 11: 'g8', 12: 'a8', 13: 'b8'}

# 2. 데이터 셋 생성

In [ ]:
def seq2dataset(seq, window_size):
    dataset = []
    
    for i in range(len(seq) - window_size):
        subset = seq[i: (i + window_size + 1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

In [ ]:
print(seq[:10])

In [ ]:
print(seq2dataset(seq[:10],4))

In [ ]:
dataset=seq2dataset(seq,4)

## 독립(입력)변수, 종속(출력)변수로 분리

In [ ]:
x_train = dataset[:,:-1]
y_train = dataset[:,-1]
x_train.shape, y_train.shape

In [ ]:
# 독립변수는 정규화
max_idx_value = max(code2idx.values())
max_idx_value

In [ ]:
x_train = x_train/max_idx_value
x_train[:2]

In [ ]:
#종속변수의 원핫인코딩
y_train = utils.to_categorical(y_train)
y_train.shape

In [ ]:
one_hot_vec_size = y_train.shape[1] # 종속변수 갯수
one_hot_vec_size

# 3. DNN

In [ ]:
# 1. 모델 생성
model = Sequential()
model.add(Dense(units=128, input_dim=4, activation='relu'))
model.add(Dense(
    units=one_hot_vec_size, 
    activation='softmax'
))
# 2. 모델 학습과정 설정
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
# 3. 모델 학습시키기
class LossHistory(Callback):
    def __init__(self):
        self.epoch = 0
        self.losses = []
        self.accuracy = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        if self.epoch % 100 == 0:
            print("epoch: {0} - loss: {1:8.6f}, acc : {2:8.6f}".\
                  format(self.epoch, logs.get('loss'), logs.get('accuracy')))
        self.epoch += 1
history = LossHistory()
for epoch in range(1000):
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=0,
             callbacks=[history])
    if (history.losses[-1] < 1e-4) & (history.accuracy[-1] > 0.99):
        print("epoch:{} - loss:{}, acc:{} - 종료".\
              format(epoch, history.losses[-1], history.accuracy[-1]))

# 4. LSTM

In [ ]:
del model, history

In [ ]:
x_train.shape

In [ ]:
x_train=x_train.reshape(50,4,1)

In [ ]:
# 1. 모델 생성
model = Sequential()
model.add(LSTM(
    units=128,
    kernel_initializer='glorot_normal',
    bias_initializer='zero',
    batch_input_shape=(1, 4, 1), 
    stateful=True
))
model.add(Dense(
    units=one_hot_vec_size, 
    kernel_initializer='glorot_normal',
    bias_initializer='zero',    
    activation='softmax'
))
# 2. 모델 학습과정 설정
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
# 3. 모델 학습시키기
class LossHistory(Callback):
    def __init__(self):
        self.epoch = 0
        self.losses = []
        self.accuracy = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracy.append(logs.get('accuracy'))
        if self.epoch % 100 == 0:
            print("epoch: {0} - loss: {1:8.6f}, acc : {2:8.6f}".\
                  format(self.epoch, logs.get('loss'), logs.get('accuracy')))
        self.epoch += 1
history = LossHistory()
for epoch in range(1000):
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=0, shuffle=False,
             callbacks=[history])
    if (history.losses[-1] < 1e-5) & (history.accuracy[-1] > 0.99):
        print("epoch:{} - loss:{}, acc:{} - 종료".\
              format(epoch, history.losses[-1], history.accuracy[-1]))
        break;
    model.reset_states() # 훈련할 때마다 이전 데이터와 상관없이 다른 데이터라고 판단

# 5. 모델 사용하기

In [ ]:
# 한 스텝 예측
seq_out = ['g8', 'e8', 'e4', 'f8']
sample = [code2idx[item] for item in seq_out]
sample = np.array(sample).reshape(1,4,1)
idx = model.predict(sample).argmax()
idx2code[idx]

In [ ]:
pred_count=50
seq_in = ['g8', 'c8', 'f4', 'e8']
seq_out = seq_in
seq_in = [code2idx[note] / float(max_idx_value) for note in seq_in]  # 코드를 인덱스값으로 변환

for i in range(pred_count):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1,4,1))  # 샘플 수, 타입스텝 수, 속성 수
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)
    seq_out.append(idx2code[idx])
    seq_in.append(idx / float(max_idx_value))
    seq_in.pop(0)

model.reset_states()

print("full song prediction : ")

for note in seq_out:
    print(note, end=" ")

In [ ]:
note_seq = ' '.join(seq_out)
note_seq

In [ ]:
m = music21.converter.parse("2/4 " + note_seq, format='tinyNotation')

m.show("midi")